# Web Scraping

· Crawler with selenium all listings for Zurich from homegate.ch

· Save the crawled data in the database

· Make sure that no duplicates make it into the database

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine, inspect, MetaData, select, Table
from selenium.webdriver.chrome.options import Options
from sqlalchemy.exc import IntegrityError
from random import randint
from time import sleep
import pandas as pd
import datetime

# Crawler methods

In [2]:
def get_next_url(url):
    """
        Increase page number by 1.
        
        https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep=9
        https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep=10
        ...
        https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep=n
    """
    split_ = url.split('=')
    return split_[0] + '=' + str(int(split_[1]) + 1)

In [3]:
def get_page(url): 
    sleep(randint(1,5)) # Random sleep time to avoid CAPTCHA and access denied
    driver.get(url)

In [4]:
def get_page_links():
    """
        Older pages have stored the links to houses in different html class names. In this cases None will be in the list of links.
        For this we check for the old class name.
    """
    links = [link.get_attribute('href') for link in driver.find_elements(By.CLASS_NAME, "ResultList_ListItem_3AwDq")]
    if None in links:
        links_old = links + [link.get_attribute('href') for link in driver.find_elements(By.CLASS_NAME, "ListItem_itemLink_30Did")]
        links = [link for link in links_old if link is not None]
    return links

In [5]:
def get_price_rooms_area():
    """
        Get price, number of rooms and living area of the house.
        In case some are not provided they are replaced by None.
    """
    obj = driver.find_elements(By.CLASS_NAME, "SpotlightAttributes_value_2njuM")
    labels = driver.find_elements(By.CLASS_NAME, "SpotlightAttributes_label_3ETFE")
    price, rooms, size = None, None, None
    for pos,label in enumerate(labels):
        if label.text == 'Rent':
            try:
                price = float(obj[pos].text[4:obj[pos].text.index('.')].replace(",",""))
            except:
                price = None  # Price on request
        elif label.text == 'Living space':   
            size = float(obj[pos].text.split()[0])  
        elif label.text == 'Rooms':
            rooms = float(obj[pos].text)
        else:
            print(label.text)
    return price, rooms, size

In [6]:
def get_id(url):
    """
        Can be found in the url -> https://www.homegate.ch/rent/3002168279
        Id = 3002168279
    """
    return url[url.index('rent') + 5:]

In [7]:
def get_address():
    """
        Get the address. Always provided.
    """
    return driver.find_elements(By.CLASS_NAME, "AddressDetails_address_3Uq1m")[0].text

In [8]:
def get_extraction_date():
    """
        Extraction date is at the moment the crawler accesses the webpage.
        Format: 2022-11-14 16:00:03.103544
    """
    return str(datetime.datetime.now())

In [9]:
def get_attributes(url):
    """
        Get id, price, rooms, area, address and extraction date
    """
    price, rooms, size = get_price_rooms_area()
    id_ = get_id(url)
    address = get_address()
    extraction_date = get_extraction_date()
    safe_to_database(id_, price, size, rooms, address, extraction_date)
    print(id_, price, rooms, size, address, extraction_date)

In [10]:
def explore_links(links):
    for link in links:
        print(link)
        get_page(link)
        get_attributes(link)

# Connection to database

In [11]:
# Using sqlalchemy to load the database
engine = create_engine('sqlite:///listing_database.db')
inspector = inspect(engine)
connection = engine.connect()
metadata = MetaData()
table_name = inspector.get_table_names()[0]
print(table_name)

listings


In [12]:
listings = Table(table_name, metadata, autoload=True, autoload_with=engine)
print(listings.columns.keys())

['listing_id', 'price', 'size', 'rooms', 'address', 'extraction_date']


In [13]:
metadata.tables

FacadeDict({'listings': Table('listings', MetaData(), Column('listing_id', INTEGER(), table=<listings>, primary_key=True), Column('price', REAL(), table=<listings>), Column('size', REAL(), table=<listings>), Column('rooms', REAL(), table=<listings>), Column('address', TEXT(), table=<listings>), Column('extraction_date', TEXT(), table=<listings>), schema=None)})

In [38]:
query = select([listings]) 
result = connection.execute(query).fetchall()
df = pd.DataFrame(result)
df

,0,1,2,3,4,5
0,3002140953,4125.0,90.0,4.5,"Waserstrasse, 8053 Zurich",2022-11-14 16:00:03.103544
1,3002206075,1080.0,26.0,1.5,"Weite Gasse, 8001 Zürich",2022-11-14 16:00:03.103544


In [14]:
def safe_to_database(id_, price, size, rooms, address, extraction_date):
    try:
        insert = metadata.tables['listings'].insert().values(listing_id=id_, price=price, size=size, rooms=rooms, address=address, extraction_date=extraction_date)
        connection.execute(insert)
    except IntegrityError:
        pass # Object already in database
    except Exception:
        print("FAILURE: ", id_, price, size, rooms, address, extraction_date)

# Crawler

In [15]:
def crawler(target_url,count=None):
    if count is None:    
        explore = 1
    else:
        explore = count
    while explore > 0:
        print("target: " + target_url)
        get_page(target_url)
        links = get_page_links()
        if len(links) == 0:
            break
        explore_links(links)
        target_url = get_next_url(target_url)
        if count:
            explore-=1

In [16]:
target_url = "https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep=1"
driver = webdriver.Chrome() # Select driver Chrome
crawler(target_url,count=1)
driver.close()

target: https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?ep=1
https://www.homegate.ch/rent/3002184266
3002184266 4250.0 2.5 70.0 Ankerstrasse, 8004 Zürich 2022-11-29 12:02:15.873120
https://www.homegate.ch/rent/3002184264
3002184264 4400.0 2.5 71.0 Ankerstrasse 114, 8004 Zürich 2022-11-29 12:02:21.107785
https://www.homegate.ch/rent/3002184262
3002184262 5700.0 3.5 94.0 Ankerstrasse 114, 8004 Zürich 2022-11-29 12:02:29.782107
https://www.homegate.ch/rent/3002184263
3002184263 5800.0 3.5 97.0 Ankerstrasse 114, 8004 Zürich 2022-11-29 12:02:35.089566
https://www.homegate.ch/rent/3002228879
3002228879 2080.0 3.0 75.0 Birmensdorferstrasse 588, 8055 Zürich 2022-11-29 12:02:40.194790
https://www.homegate.ch/rent/3002140953
3002140953 4125.0 4.5 90.0 Waserstrasse, 8053 Zurich 2022-11-29 12:02:46.179520
https://www.homegate.ch/rent/3002253311
3002253311 1000.0 2.5 78.0 Froschaugasse 18, 8001 Zürich 2022-11-29 12:02:48.982204
https://www.homegate.ch/rent/3002104105
3002104105 2

# Delete new inserted rows

In [41]:
#stmt=metadata.tables['listings'].delete().where(metadata.tables['listings'].c.extraction_date > "2022-11-15 16:00:03.103544")
#connection.execute(stmt)

# Challenges

To many requests, saturating the server:

<img src="../img/captcha1.png" width="600">

Different User-Agents get denied access:

<img src="../img/denied1.png" width="600">

Different html structures:

<img src="../img/rent.png" width="600">
<img src="../img/rent_all.png" width="600">
<img src="../img/rent_week.png" width="600">